## Basic Exploration of the Daily Dataset

The objective of this notebook is to find problems in the data set of daily history and posible cleaup strategies before doing the full analysis.

### Libraries used in this analysis

It will be mostly executed in spark but will do some local processing

In [1]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import Row

### PWS Information and Globals

This is the infromation of the sensor that performed the measurements. The ID is the one given by wunderground. In this case we use only cinvestav telchac.

In [2]:
#PWS info
pwsID = 'IYUCATNT2'
pwsTz = 'America/Merida'

#Global info of the analysis
startDate='2012-01-01'
endDate='2016-12-31'

### Reading the dataset

The data set is saved in a hierarchical maner having independent directories for year and subdirectories for each month.

In [3]:
basePath = 'dailyHistory/'+pwsID+'/'
datePath='%Y/%m/'

dirDateRange = pd.date_range(start=startDate,end=endDate,freq='MS',tz=pwsTz)

dailyHistoryFiles = map(lambda ts:  basePath+ts.strftime(datePath)+'*.csv',dirDateRange)

# This line consumes the generator and it gets empty for future call
dailyHistoryFilesList = list(dailyHistoryFiles)
dailyHistoryFilesList

['dailyHistory/IYUCATNT2/2012/01/*.csv',
 'dailyHistory/IYUCATNT2/2012/02/*.csv',
 'dailyHistory/IYUCATNT2/2012/03/*.csv',
 'dailyHistory/IYUCATNT2/2012/04/*.csv',
 'dailyHistory/IYUCATNT2/2012/05/*.csv',
 'dailyHistory/IYUCATNT2/2012/06/*.csv',
 'dailyHistory/IYUCATNT2/2012/07/*.csv',
 'dailyHistory/IYUCATNT2/2012/08/*.csv',
 'dailyHistory/IYUCATNT2/2012/09/*.csv',
 'dailyHistory/IYUCATNT2/2012/10/*.csv',
 'dailyHistory/IYUCATNT2/2012/11/*.csv',
 'dailyHistory/IYUCATNT2/2012/12/*.csv',
 'dailyHistory/IYUCATNT2/2013/01/*.csv',
 'dailyHistory/IYUCATNT2/2013/02/*.csv',
 'dailyHistory/IYUCATNT2/2013/03/*.csv',
 'dailyHistory/IYUCATNT2/2013/04/*.csv',
 'dailyHistory/IYUCATNT2/2013/05/*.csv',
 'dailyHistory/IYUCATNT2/2013/06/*.csv',
 'dailyHistory/IYUCATNT2/2013/07/*.csv',
 'dailyHistory/IYUCATNT2/2013/08/*.csv',
 'dailyHistory/IYUCATNT2/2013/09/*.csv',
 'dailyHistory/IYUCATNT2/2013/10/*.csv',
 'dailyHistory/IYUCATNT2/2013/11/*.csv',
 'dailyHistory/IYUCATNT2/2013/12/*.csv',
 'dailyHistory/I

Now that we have the list of files we proceed to open them as spark dataset.

It looks like some files came empty at the time of querying the url. It looks like this causes the infer schema to fail. Everything will be a string.

The problem with calling the load with the result of the map is that it is a generator and it was being consumed by the list() operation before. When called in the load it was empty. A generator can't be used 2 times. In order to be reusable I explicitly converted to list in the previous cell.

In [4]:
# Changing the default timezone for the analysis
spark.conf.set("spark.sql.session.timeZone", pwsTz)

# Inferr schema creates a problem when changing the session timezone
#    .option("inferSchema","true") \
    
staticDataFrame = spark.read \
    .format("csv") \
    .option("header","true") \
    .load(dailyHistoryFilesList)
#    .load('dailyHistory/IYUCATNT2/2016/01/IYUCATNT2-2016-01-02.csv')
#    .load('dailyHistory/IYUCATNT2/*/*/*.csv')
#    .load(list(dailyHistoryFiles))
#    .load(['dailyHistory/IYUCATNT2/2016/01/*.csv',
# 'dailyHistory/IYUCATNT2/2016/02/*.csv'])
    #.load('dailyHistory/IYUCATNT2/2016/*/*.csv')
    #.load(list(dailyHistoryFiles))

staticDataFrame.printSchema()
staticDataFrame.show()

root
 |-- Time: string (nullable = true)
 |-- TemperatureF: string (nullable = true)
 |-- DewpointF: string (nullable = true)
 |-- PressureIn: string (nullable = true)
 |-- WindDirection: string (nullable = true)
 |-- WindDirectionDegrees: string (nullable = true)
 |-- WindSpeedMPH: string (nullable = true)
 |-- WindSpeedGustMPH: string (nullable = true)
 |-- Humidity: string (nullable = true)
 |-- HourlyPrecipIn: string (nullable = true)
 |-- Conditions: string (nullable = true)
 |-- Clouds: string (nullable = true)
 |-- dailyrainin: string (nullable = true)
 |-- SolarRadiationWatts/m^2: string (nullable = true)
 |-- SoftwareType: string (nullable = true)
 |-- DateUTC: string (nullable = true)

+-------------------+------------+---------+----------+-------------+--------------------+------------+----------------+--------+--------------+----------+------+-----------+-----------------------+-------------------+-------------------+
|               Time|TemperatureF|DewpointF|PressureIn|W

## Look for low samples or Zero Sample Dates

We will see how many readings we have per day and if there are null days (will be substitute by 0).

First creating a data frame with all the dates possible in the range.

In [5]:
allDateRange = pd.date_range(start=startDate,end=endDate,freq='D',tz=pwsTz)
# Range to RDD of stringd
allDateStrRDD = sc.parallelize(allDateRange).map(lambda ts: ts.strftime('%Y-%m-%d'))
# RDD of strings to df of dates
allDateDF = allDateStrRDD.map(Row("Date")).toDF().selectExpr("cast(Date as date)")

allDateDF.printSchema()

root
 |-- Date: date (nullable = true)



Now converting the time colum to timestamp

In [6]:
tsDateDF = staticDataFrame \
    .selectExpr("to_timestamp(Time,'yyyy-MM-dd HH:mm:ss') as TS") \
    .withColumn("Date",col("TS").cast("date"))
tsDateDF.printSchema()

root
 |-- TS: timestamp (nullable = true)
 |-- Date: date (nullable = true)



In [7]:
dateCountsDF = tsDateDF.groupBy("Date").count() #.sort("count")
size = dateCountsDF.count()
dateCountsDF.show(size)

+----------+-----+
|      Date|count|
+----------+-----+
|2013-05-21|  195|
|2013-01-22|  196|
|2012-10-06|  196|
|2013-09-09|  190|
|2016-03-01|  188|
|2015-05-19|  184|
|2015-03-09|  182|
|2014-09-26|  181|
|2013-03-26|  177|
|2014-11-12|  174|
|2013-09-19|  189|
|2015-03-06|  187|
|2016-04-25|  183|
|2016-05-03|  280|
|2016-08-31|  275|
|2015-12-22|  224|
|2016-08-15|  222|
|2014-08-01|  191|
|2015-04-09|  190|
|2012-11-11|  187|
|2015-09-02|  186|
|2016-10-03|  178|
|2013-02-02|  178|
|2016-01-28|  211|
|2012-02-12|  205|
|2014-06-03|  190|
|2012-08-23|  173|
|2016-07-17|  249|
|2014-02-16|  194|
|2014-06-11|  190|
|2015-05-10|  188|
|2012-07-14|  189|
|2013-09-12|  186|
|2015-09-28|  185|
|2013-02-04|  185|
|2014-01-24|  180|
|2016-11-08|  176|
|2016-12-19|  173|
|2012-03-30|   42|
|2015-03-16|  229|
|2015-04-24|  211|
|2012-11-03|  202|
|2012-09-11|  197|
|2014-02-18|  192|
|2015-04-01|  189|
|2013-11-18|  190|
|2015-03-12|  187|
|2014-08-13|  185|
|2016-08-23|  181|
|2016-07-03|

Join all dates dataset with the counts

In [8]:
# Using a sequencer gives better results since there is just one Date column
joinDF = allDateDF.join(dateCountsDF,["Date"],"left_outer").na.fill({'count':0})


size = joinDF.count()
joinDF.sort("count").show(size)



+----------+-----+
|      Date|count|
+----------+-----+
|2015-05-24|    0|
|2016-03-19|    0|
|2016-06-09|    0|
|2015-05-02|    0|
|2016-06-10|    0|
|2016-01-09|    0|
|2016-03-21|    0|
|2016-06-13|    0|
|2016-01-06|    0|
|2016-06-20|    0|
|2016-03-26|    0|
|2016-07-26|    0|
|2016-08-21|    0|
|2016-01-13|    0|
|2016-03-30|    0|
|2016-08-27|    0|
|2016-11-03|    0|
|2016-03-31|    0|
|2016-11-06|    0|
|2015-06-27|    0|
|2016-11-14|    0|
|2016-01-16|    0|
|2016-06-05|    0|
|2016-06-06|    0|
|2016-01-03|    0|
|2016-01-08|    0|
|2016-06-08|    0|
|2012-04-09|    0|
|2012-09-21|    0|
|2012-05-01|    0|
|2012-09-22|    0|
|2012-09-23|    0|
|2012-04-17|    0|
|2012-05-02|    0|
|2012-09-24|    0|
|2012-09-25|    0|
|2012-05-03|    0|
|2012-09-26|    0|
|2012-09-27|    0|
|2012-04-18|    0|
|2012-05-04|    0|
|2012-09-28|    0|
|2012-04-01|    0|
|2012-04-10|    0|
|2012-09-29|    0|
|2012-05-05|    0|
|2012-09-30|    0|
|2012-10-01|    0|
|2012-04-19|    0|
|2012-05-06|

## Exploring the Wind Speed and Wind Direction

Now let's look at the ranges in the wind speed and wind directon to see what we got.

We will be dealing with this data, this is the inferred schema of a good one:

 |-- Time: timestamp (nullable = true)

 |-- WindDirection: string (nullable = true)
 
 |-- WindDirectionDegrees: integer (nullable = true)
 
 |-- WindSpeedMPH: double (nullable = true)
 
 |-- WindSpeedGustMPH: double (nullable = true)
 
 

In [9]:
windDF = staticDataFrame.selectExpr(
    "Time",
    "to_timestamp(Time,'yyyy-MM-dd HH:mm:ss') as TS",
    "WindDirection",
    "cast(WindDirectionDegrees as integer)",
    "cast(WindSpeedMPH as double)",
    "cast(WindSpeedGustMPH as double)")
# Creating SQL for future analisys
windDF.createOrReplaceTempView("windTable")
windDF.printSchema()
windDF.show()

root
 |-- Time: string (nullable = true)
 |-- TS: timestamp (nullable = true)
 |-- WindDirection: string (nullable = true)
 |-- WindDirectionDegrees: integer (nullable = true)
 |-- WindSpeedMPH: double (nullable = true)
 |-- WindSpeedGustMPH: double (nullable = true)

+-------------------+-------------------+-------------+--------------------+------------+----------------+
|               Time|                 TS|WindDirection|WindDirectionDegrees|WindSpeedMPH|WindSpeedGustMPH|
+-------------------+-------------------+-------------+--------------------+------------+----------------+
|2016-05-10 00:00:00|2016-05-10 00:00:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:05:00|2016-05-10 00:05:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:10:00|2016-05-10 00:10:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:15:00|2016-05-10 00:15:00|          N/A|             -737280|         0.0|

Summary statistics:

In [10]:
windDF.describe().show()

+-------+-------------------+-------------+--------------------+-----------------+------------------+
|summary|               Time|WindDirection|WindDirectionDegrees|     WindSpeedMPH|  WindSpeedGustMPH|
+-------+-------------------+-------------+--------------------+-----------------+------------------+
|  count|             322185|       322185|              322185|           322185|            322185|
|   mean|               null|         null| -1925.1210764002049|9.334107112373326|-86.54209848379037|
| stddev|               null|         null|   38843.62690559665|28.01871695736958|298.68542230734556|
|    min|2012-01-01 00:04:00|          ENE|             -737280|           -999.9|            -999.0|
|    max|2016-12-31 23:52:00|         West|               32767|             41.0|              46.0|
+-------+-------------------+-------------+--------------------+-----------------+------------------+



There are some values that need to be investigated in the dataset. There is a very high and verylow degrees in WindDirectionDegrees. Let's see how they look like.

### Odd Degrees

These are the 32767 and -737280 degrees:

In [11]:
spark.sql("""
SELECT *
FROM windTable
WHERE WindDirectionDegrees=32767
""").show()

+-------------------+-------------------+-------------+--------------------+------------+----------------+
|               Time|                 TS|WindDirection|WindDirectionDegrees|WindSpeedMPH|WindSpeedGustMPH|
+-------------------+-------------------+-------------+--------------------+------------+----------------+
|2013-03-14 00:00:00|2013-03-14 00:00:00|        North|               32767|      -999.0|             0.0|
|2013-03-14 00:05:00|2013-03-14 00:05:00|        North|               32767|      -999.0|             0.0|
|2013-03-14 00:10:00|2013-03-14 00:10:00|        North|               32767|      -999.0|             0.0|
|2013-03-14 00:20:00|2013-03-14 00:20:00|        North|               32767|      -999.0|             0.0|
|2013-03-14 00:25:00|2013-03-14 00:25:00|        North|               32767|      -999.0|             0.0|
|2013-03-14 00:35:00|2013-03-14 00:35:00|        North|               32767|      -999.0|             0.0|
|2013-03-14 00:40:00|2013-03-14 00:40

In [12]:
spark.sql("""
SELECT *
FROM windTable
WHERE WindDirectionDegrees=-737280
""").show(100)

+-------------------+-------------------+-------------+--------------------+------------+----------------+
|               Time|                 TS|WindDirection|WindDirectionDegrees|WindSpeedMPH|WindSpeedGustMPH|
+-------------------+-------------------+-------------+--------------------+------------+----------------+
|2016-05-10 00:00:00|2016-05-10 00:00:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:05:00|2016-05-10 00:05:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:10:00|2016-05-10 00:10:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:15:00|2016-05-10 00:15:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:20:00|2016-05-10 00:20:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:25:00|2016-05-10 00:25:00|          N/A|             -737280|         0.0|          -999.0|
|2016-05-10 00:30:00|2016-05-10 00:30

It looks like the very low -737280 means there is a a windspeed of 0 not a big deal. In case of the windspeed of 32767 there is very few and looks like some sort of NA.

So now let's see the frequency of distinct values that are not inside the 0 360 degrees.

In [13]:
spark.sql("""
SELECT WindDirectionDegrees, COUNT(WindDirectionDegrees)
FROM windTable
WHERE WindDirectionDegrees NOT BETWEEN 0 and 360
GROUP BY WindDirectionDegrees
""").show()

+--------------------+---------------------------+
|WindDirectionDegrees|count(WindDirectionDegrees)|
+--------------------+---------------------------+
|               32767|                        242|
|             -737280|                        896|
+--------------------+---------------------------+



Now look at all possible combinations

In [14]:
spark.sql("""
SELECT DISTINCT WindDirection, WindDirectionDegrees, WindSpeedMPH, WindSpeedGustMPH
FROM windTable
WHERE WindDirectionDegrees NOT BETWEEN 0 and 360
""").show()

+-------------+--------------------+------------+----------------+
|WindDirection|WindDirectionDegrees|WindSpeedMPH|WindSpeedGustMPH|
+-------------+--------------------+------------+----------------+
|        North|               32767|         9.0|            12.0|
|        North|               32767|      -999.9|             0.0|
|          N/A|             -737280|         0.0|          -999.0|
|        North|               32767|      -999.0|             0.0|
|        North|               32767|         0.0|             0.0|
|        North|               32767|         1.0|             6.0|
+-------------+--------------------+------------+----------------+



Now excluding the angles and get another summary

In [15]:
spark.sql("""
SELECT *
FROM windTable
WHERE WindDirectionDegrees BETWEEN 0 AND 360
""").describe().show()

+-------+-------------------+-------------+--------------------+------------------+------------------+
|summary|               Time|WindDirection|WindDirectionDegrees|      WindSpeedMPH|  WindSpeedGustMPH|
+-------+-------------------+-------------+--------------------+------------------+------------------+
|  count|             321047|       321047|              321047|            321047|            321047|
|   mean|               null|         null|  101.00742881883338|10.104641999458023|-84.06083844421534|
| stddev|               null|         null|   72.37711508012066| 6.033940030117123|295.28591571438926|
|    min|2012-01-01 00:04:00|          ENE|                   0|               0.0|            -999.0|
|    max|2016-12-31 23:52:00|         West|                 360|              41.0|              46.0|
+-------+-------------------+-------------+--------------------+------------------+------------------+



The only negative number still remains the negative windgust. Let's find out what is that

In [16]:
spark.sql("""
SELECT *
FROM windTable
WHERE WindSpeedGustMPH < 0 AND WindDirectionDegrees BETWEEN 0 AND 360
""").show(100)

+-------------------+-------------------+-------------+--------------------+------------+----------------+
|               Time|                 TS|WindDirection|WindDirectionDegrees|WindSpeedMPH|WindSpeedGustMPH|
+-------------------+-------------------+-------------+--------------------+------------+----------------+
|2016-05-10 08:35:00|2016-05-10 08:35:00|           SE|                 135|         0.0|          -999.0|
|2016-05-10 08:40:00|2016-05-10 08:40:00|           SE|                 135|         0.0|          -999.0|
|2016-05-10 18:45:00|2016-05-10 18:45:00|          ENE|                  68|         8.0|          -999.0|
|2016-05-10 18:50:00|2016-05-10 18:50:00|          ENE|                  68|         8.0|          -999.0|
|2016-05-10 18:55:00|2016-05-10 18:55:00|          ENE|                  68|        17.0|          -999.0|
|2016-05-10 19:00:00|2016-05-10 19:00:00|          ENE|                  68|        17.0|          -999.0|
|2016-05-10 19:05:00|2016-05-10 19:05

In [17]:
spark.sql("""
SELECT DISTINCT WindDirection, WindDirectionDegrees, WindSpeedMPH, WindSpeedGustMPH
FROM windTable
WHERE WindSpeedGustMPH < 0 AND WindDirectionDegrees BETWEEN 0 AND 360
""").show(100)

+-------------+--------------------+------------+----------------+
|WindDirection|WindDirectionDegrees|WindSpeedMPH|WindSpeedGustMPH|
+-------------+--------------------+------------+----------------+
|          NNW|                 338|        30.0|          -999.0|
|         East|                  90|        19.0|          -999.0|
|          NNW|                 338|        14.0|          -999.0|
|          WSW|                 248|         1.0|          -999.0|
|          ENE|                  68|        12.0|          -999.0|
|          ENE|                  68|         4.0|          -999.0|
|          SSE|                 158|         9.0|          -999.0|
|         East|                  90|         5.0|          -999.0|
|          WNW|                 293|        24.0|          -999.0|
|          ESE|                 113|         7.0|          -999.0|
|          WNW|                 293|         2.0|          -999.0|
|         East|                  90|        21.0|          -99

It is really common and it is not repeated too much. We are not taking into account the gust now but otherwise this is candidate for N/A.

Next, let's find out why there is Wind directions for 0 and 360.

In [18]:
spark.sql("""
SELECT *
FROM windTable
WHERE WindDirectionDegrees=0 OR WindDirectionDegrees=360
""").show()

+-------------------+-------------------+-------------+--------------------+------------+----------------+
|               Time|                 TS|WindDirection|WindDirectionDegrees|WindSpeedMPH|WindSpeedGustMPH|
+-------------------+-------------------+-------------+--------------------+------------+----------------+
|2012-06-21 19:25:00|2012-06-21 19:25:00|        North|                   0|         7.0|          -999.0|
|2012-06-21 19:30:00|2012-06-21 19:30:00|        North|                   0|         7.0|          -999.0|
|2012-06-21 19:35:00|2012-06-21 19:35:00|        North|                   0|         6.0|          -999.0|
|2012-06-21 19:40:00|2012-06-21 19:40:00|        North|                   0|         6.0|          -999.0|
|2016-05-05 00:00:00|2016-05-05 00:00:00|        North|                   0|        15.0|          -999.0|
|2016-05-05 00:05:00|2016-05-05 00:05:00|        North|                   0|        15.0|          -999.0|
|2016-05-05 00:10:00|2016-05-05 00:10

The distribution of 0 and 360 looks like:

In [19]:
spark.sql("""
SELECT WindDirection, WindDirectionDegrees, COUNT(1)
FROM windTable
WHERE WindDirectionDegrees=0 OR WindDirectionDegrees=360
GROUP BY WindDirection, WindDirectionDegrees
""").show()

+-------------+--------------------+--------+
|WindDirection|WindDirectionDegrees|count(1)|
+-------------+--------------------+--------+
|        North|                 360|    2557|
|        North|                   0|    1497|
+-------------+--------------------+--------+



Now extracting north only and see how it looks like

In [20]:
spark.sql("""
SELECT *
FROM windTable
WHERE WindDirection='North' AND WindDirectionDegrees BETWEEN 0 AND 360
""").describe().show()

+-------+-------------------+-------------+--------------------+------------------+-------------------+
|summary|               Time|WindDirection|WindDirectionDegrees|      WindSpeedMPH|   WindSpeedGustMPH|
+-------+-------------------+-------------+--------------------+------------------+-------------------+
|  count|              13145|        13145|               13145|             13145|              13145|
|   mean|               null|         null|   165.6192468619247|13.930239634842145|-100.58851274248764|
| stddev|               null|         null|  174.61484773493234| 6.049684162424559|  322.1444868292934|
|    min|2012-01-02 10:39:00|        North|                   0|               0.0|             -999.0|
|    max|2016-12-31 10:27:00|        North|                 360|              41.0|               46.0|
+-------+-------------------+-------------+--------------------+------------------+-------------------+



In [21]:
spark.sql("""
SELECT WindDirection, WindDirectionDegrees, COUNT(1)
FROM windTable
WHERE WindDirection='North' AND WindDirectionDegrees BETWEEN 0 AND 360
GROUP BY WindDirection, WindDirectionDegrees
""").show()

+-------------+--------------------+--------+
|WindDirection|WindDirectionDegrees|count(1)|
+-------------+--------------------+--------+
|        North|                 349|     232|
|        North|                 360|    2557|
|        North|                   5|    3000|
|        North|                   0|    1497|
|        North|                 350|     491|
|        North|                  11|     982|
|        North|                 355|    2418|
|        North|                 351|     296|
|        North|                  10|     591|
|        North|                   9|    1081|
+-------------+--------------------+--------+



#### Bad Average Around North

The extraction of the North degrees show there is a problem when averaging them, the average is not in the north and it should be. Min and max are useless since it covers all the range.

In [22]:
spark.sql("""
SELECT AVG(WindDirectionDegrees), MIN(WindDirectionDegrees), MAX(WindDirectionDegrees)
FROM windTable
WHERE WindDirection='North' AND WindDirectionDegrees BETWEEN 0 AND 360
""").show()

+-------------------------+-------------------------+-------------------------+
|avg(WindDirectionDegrees)|min(WindDirectionDegrees)|max(WindDirectionDegrees)|
+-------------------------+-------------------------+-------------------------+
|        165.6192468619247|                        0|                      360|
+-------------------------+-------------------------+-------------------------+



This is an attempt to calculate the average by making high angles negatives:

In [23]:
spark.sql("""
SELECT AVG(Angle), MIN(Angle), MAX(Angle)
FROM (
  SELECT
    WindDirectionDegrees,
    CASE WHEN WindDirectionDegrees > 180 THEN WindDirectionDegrees-360 ELSE WindDirectionDegrees END AS Angle     
  FROM windTable
  WHERE WindDirection='North' AND WindDirectionDegrees BETWEEN 0 AND 360
)
""").show()

+------------------+----------+----------+
|        avg(Angle)|min(Angle)|max(Angle)|
+------------------+----------+----------+
|1.4625332826169646|       -11|        11|
+------------------+----------+----------+



Now let's calculate the X and Y components and use that to average. From 2012 to 2015 the average was 1.4625332826169646.

Some experiments about the angles

In [40]:
angleDf = spark.sql("""
SELECT WindDirection, WindDirectionDegrees, COUNT(1) as SampleCount
FROM windTable
WHERE WindDirection='North' AND WindDirectionDegrees BETWEEN 0 AND 360
GROUP BY WindDirection, WindDirectionDegrees
""")

angleDf.show()

recDf = angleDf \
    .withColumn("rad",radians("WindDirectionDegrees")) \
    .withColumn("x",cos("rad")) \
    .withColumn("y",sin("rad"))

recDf.show(truncate=False)

recDf.selectExpr("sum(x)","sum(y)").show(truncate=False)

+-------------+--------------------+-----------+
|WindDirection|WindDirectionDegrees|SampleCount|
+-------------+--------------------+-----------+
|        North|                 349|        232|
|        North|                 360|       2557|
|        North|                   5|       3000|
|        North|                   0|       1497|
|        North|                 350|        491|
|        North|                  11|        982|
|        North|                 355|       2418|
|        North|                 351|        296|
|        North|                  10|        591|
|        North|                   9|       1081|
+-------------+--------------------+-----------+

+-------------+--------------------+-----------+-------------------+------------------+-----------------------+
|WindDirection|WindDirectionDegrees|SampleCount|rad                |x                 |y                      |
+-------------+--------------------+-----------+-------------------+------------------+--

Average: 1.4625332826169646 vs 1.463993168057204, close but not the same. The result is the same if there is a SUM or AVG function in the x and y components.

In [56]:
recAvgDF = spark.sql("""
SELECT SUM(x) AS avgX, SUM(y) AS avgY, degrees(atan2(sum(y),sum(x)))
FROM (
  SELECT
    WindDirectionDegrees,
    cos(radians(WindDirectionDegrees)) AS x,
    sin(radians(WindDirectionDegrees)) AS y 
  FROM windTable
  WHERE WindDirection='North' AND WindDirectionDegrees BETWEEN 0 AND 360
)
""")#.withColumn("avgWindDirection",expr("degrees(atan2(avgY,avgX))"))

recAvgDF.show(truncate=False)


+------------------+-----------------+------------------------------+
|avgX              |avgY             |DEGREES(ATAN2(sum(y), sum(x)))|
+------------------+-----------------+------------------------------+
|13068.687108725273|333.9972616348193|1.463993168057204             |
+------------------+-----------------+------------------------------+

